In [19]:
import torch
import norse
import snntorch
import snntorch.import_nirtorch
import nir
import importlib
importlib.reload(norse)

<module 'norse' from '/home/jens/work/norse/norse/__init__.py'>

In [65]:
nir_graph = nir.read("braille_noDelay_bias_zero.nir")
net = snntorch.import_nirtorch.from_nir(nir_graph)

replace rnn subgraph with nirgraph


In [98]:
test_data_path = "data/ds_test.pt"
ds_test = torch.load(test_data_path)
data = net.fc1(ds_test[0][0])
data.shape

torch.Size([256, 38])

In [101]:
c = net.lif1
spk, syn, mem = c.init_rsynaptic()
# c.recurrent.bias.data.fill_(0)
c

RSynaptic(
  (recurrent): Linear(in_features=38, out_features=38, bias=True)
)

In [102]:
spk, syn, mem = c(data[0], spk, syn, mem)
spk, syn, mem

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        grad_fn=<MulBackward0>),
 tensor([ 0.0650, -0.3780,  0.3210, -0.8547, -1.2234, -0.0975, -0.4730, -1.8350,
         -1.5277, -3.3729,  0.0066, -1.8141, -0.4551, -0.3952,  0.1220, -0.4740,
         -0.4608, -0.6093, -2.6107,  0.2287, -1.1192, -2.6595, -0.8132, -0.4155,
         -0.8495,  0.1084, -0.7751, -0.0655, -2.4296, -0.4946, -1.4717, -0.6609,
          0.1681,  0.2094, -0.5678, -0.6838, -0.3878,  0.1772],
        grad_fn=<SubBackward0>),
 tensor([ 0.0650, -0.3780,  0.3210, -0.8547, -1.2234, -0.0975, -0.4730, -1.8350,
         -1.5277, -3.3729,  0.0066, -1.8141, -0.4551, -0.3952,  0.1220, -0.4740,
         -0.4608, -0.6093, -2.6107,  0.2287, -1.1192, -2.6595, -0.8132, -0.4155,
         -0.8495,  0.1084, -0.7751, -0.0655, -2.4296, -0.4946, -1.4717, -0.6609,
          0.1681,  0.2094, -0.5678, -0.6838, -0.387

In [103]:
spk, syn, mem = c(data[1], spk, syn, mem)
spk, syn, mem

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        grad_fn=<MulBackward0>),
 tensor([ 0.1008, -0.5860,  0.4976, -1.3248, -1.8963, -0.1512, -0.7332, -2.8443,
         -2.3679, -5.2280,  0.0103, -2.8119, -0.7054, -0.6125,  0.1890, -0.7348,
         -0.7142, -0.9445, -4.0467,  0.3545, -1.7347, -4.1222, -1.2604, -0.6440,
         -1.3167,  0.1681, -1.2013, -0.1015, -3.7659, -0.7667, -2.2812, -1.0244,
          0.2605,  0.3246, -0.8800, -1.0598, -0.6011,  0.2747],
        grad_fn=<SubBackward0>),
 tensor([ 0.1594, -0.9262,  0.7865, -2.0941, -2.9973, -0.2389, -1.1589, -4.4958,
         -3.7427, -8.2636,  0.0162, -4.4446, -1.1150, -0.9682,  0.2988, -1.1614,
         -1.1290, -1.4929, -6.3963,  0.5603, -2.7419, -6.5157, -1.9923, -1.0180,
         -2.0812,  0.2657, -1.8989, -0.1605, -5.9526, -1.2118, -3.6058, -1.6192,
          0.4117,  0.5131, -1.3910, -1.6752, -0.950

In [68]:
dt = 1e-4
tau_syn = dt / (1 - c.alpha)
tau_mem = dt / (1 - c.beta)
r = tau_mem / dt
v_leak = torch.zeros_like(c.beta)
w_in = tau_syn / dt
ss = sm = None

In [69]:
nsyn = norse.torch.LIBoxCell(p = norse.torch.LIBoxParameters(tau_mem_inv=1/tau_syn), dt=1e-4)
nmem = norse.torch.LIFBoxCell(p = norse.torch.LIFBoxParameters(tau_mem_inv=1/tau_syn), dt=1e-4)
vs, ss = nsyn((c.recurrent.bias +torch.ones(38)) * w_in)
vs

tensor([ 1.1317,  0.6847,  1.2047,  0.6166,  0.4497,  0.9264,  0.7192,  0.2399,
         0.1050, -0.6780,  1.2016,  0.1231,  0.5748,  0.6751,  0.9058,  0.6672,
         0.6062,  0.7439, -0.2957,  1.0488,  0.5626, -0.2167,  0.6503,  0.7738,
         0.4772,  1.0948,  0.6632,  0.8930, -0.1727,  0.9268,  0.1753,  0.7222,
         1.0082,  0.9580,  0.6293,  0.5535,  0.7430,  1.1153],
       grad_fn=<AddBackward0>)

In [70]:
vm, s = nmem(vs * w_in)
vm, s
# nmem(v * r, s)

(tensor([1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 1.], grad_fn=<SuperSpikeBackward>),
 LIFBoxFeedForwardState(v=tensor([ 0.0000,  0.6847,  0.0000,  0.6166,  0.4497,  0.9264,  0.7192,  0.2399,
          0.1050, -0.6780,  0.0000,  0.1231,  0.5748,  0.6751,  0.9058,  0.6672,
          0.6062,  0.7439, -0.2957,  0.0000,  0.5626, -0.2167,  0.6503,  0.7738,
          0.4772,  0.0000,  0.6632,  0.8930, -0.1727,  0.9268,  0.1753,  0.7222,
          0.0000,  0.9580,  0.6293,  0.5535,  0.7430,  0.0000],
        grad_fn=<AddBackward0>)))